# SGLang Native APIs

Apart from the OpenAI compatible APIs, the SGLang Runtime also provides its native server APIs. We introduce these following APIs:

- `/generate` (text generation model)
- `/get_model_info`
- `/get_server_info`
- `/health`
- `/health_generate`
- `/flush_cache`
- `/update_weights`
- `/encode`(embedding model)
- `/classify`(reward model)
- `/start_expert_distribution_record`
- `/stop_expert_distribution_record`
- `/dump_expert_distribution_record`

We mainly use `requests` to test these APIs in the following examples. You can also use `curl`.

## Launch A Server

In [1]:
import requests
from sglang.test.test_utils import is_in_ci

if is_in_ci():
    from patch import launch_server_cmd
else:
    from sglang.utils import launch_server_cmd

from sglang.utils import wait_for_server, print_highlight, terminate_process


server_process, port = launch_server_cmd(
    "python3 -m sglang.launch_server --model-path qwen/qwen2.5-0.5b-instruct --host 0.0.0.0"
)

wait_for_server(f"http://localhost:{port}")

[2025-05-20 04:56:38] server_args=ServerArgs(model_path='qwen/qwen2.5-0.5b-instruct', tokenizer_path='qwen/qwen2.5-0.5b-instruct', tokenizer_mode='auto', skip_tokenizer_init=False, load_format='auto', trust_remote_code=False, dtype='auto', kv_cache_dtype='auto', quantization=None, quantization_param_path=None, context_length=None, device='cuda', served_model_name='qwen/qwen2.5-0.5b-instruct', chat_template=None, completion_template=None, is_embedding=False, enable_multimodal=None, revision=None, host='0.0.0.0', port=35814, mem_fraction_static=0.88, max_running_requests=200, max_total_tokens=20480, chunked_prefill_size=8192, max_prefill_tokens=16384, schedule_policy='fcfs', schedule_conservativeness=1.0, cpu_offload_gb=0, page_size=1, tp_size=1, pp_size=1, max_micro_batch_size=None, stream_interval=1, stream_output=False, random_seed=654772907, constrained_json_whitespace_pattern=None, watchdog_timeout=300, dist_timeout=None, download_dir=None, base_gpu_id=0, gpu_id_step=1, log_level='i

[2025-05-20 04:56:48] Attention backend not set. Use fa3 backend by default.
[2025-05-20 04:56:48] Init torch distributed begin.


[2025-05-20 04:56:49] Init torch distributed ends. mem usage=0.00 GB
[2025-05-20 04:56:49] init_expert_location from trivial


[2025-05-20 04:56:50] Load weight begin. avail mem=61.82 GB


[2025-05-20 04:56:51] Using model weights format ['*.safetensors']


...

[2025-05-20 04:56:55] max_total_num_tokens=20480, chunked_prefill_size=8192, max_prefill_tokens=16384, max_running_requests=200, context_len=32768


[2025-05-20 04:56:56] INFO:     Started server process [398325]
[2025-05-20 04:56:56] INFO:     Waiting for application startup.
[2025-05-20 04:56:56] INFO:     Application startup complete.
[2025-05-20 04:56:56] INFO:     Uvicorn running on http://0.0.0.0:35814 (Press CTRL+C to quit)


[2025-05-20 04:56:57] INFO:     127.0.0.1:43234 - "GET /v1/models HTTP/1.1" 200 OK


[2025-05-20 04:56:57] INFO:     127.0.0.1:43236 - "GET /get_model_info HTTP/1.1" 200 OK
[2025-05-20 04:56:57] Prefill batch. #new-seq: 1, #new-token: 6, #cached-token: 0, token usage: 0.00, #running-req: 0, #queue-req: 0


[2025-05-20 04:56:58] INFO:     127.0.0.1:43248 - "POST /generate HTTP/1.1" 200 OK
[2025-05-20 04:56:58] The server is fired up and ready to roll!


## Generate (text generation model)
Generate completions. This is similar to the `/v1/completions` in OpenAI API. Detailed parameters can be found in the [sampling parameters](./sampling_params.md).

In [2]:
url = f"http://localhost:{port}/generate"
data = {"text": "What is the capital of France?"}

response = requests.post(url, json=data)
print_highlight(response.json())

[2025-05-20 04:57:02] Prefill batch. #new-seq: 1, #new-token: 7, #cached-token: 0, token usage: 0.00, #running-req: 0, #queue-req: 0


[2025-05-20 04:57:02] Decode batch. #running-req: 1, #token: 40, token usage: 0.00, cuda graph: False, gen throughput (token/s): 5.99, #queue-req: 0


[2025-05-20 04:57:02] Decode batch. #running-req: 1, #token: 80, token usage: 0.00, cuda graph: False, gen throughput (token/s): 87.78, #queue-req: 0


[2025-05-20 04:57:03] Decode batch. #running-req: 1, #token: 120, token usage: 0.01, cuda graph: False, gen throughput (token/s): 91.66, #queue-req: 0
[2025-05-20 04:57:03] INFO:     127.0.0.1:37702 - "POST /generate HTTP/1.1" 200 OK


## Get Model Info

Get the information of the model.

- `model_path`: The path/name of the model.
- `is_generation`: Whether the model is used as generation model or embedding model.
- `tokenizer_path`: The path/name of the tokenizer.

In [3]:
url = f"http://localhost:{port}/get_model_info"

response = requests.get(url)
response_json = response.json()
print_highlight(response_json)
assert response_json["model_path"] == "qwen/qwen2.5-0.5b-instruct"
assert response_json["is_generation"] is True
assert response_json["tokenizer_path"] == "qwen/qwen2.5-0.5b-instruct"
assert response_json.keys() == {"model_path", "is_generation", "tokenizer_path"}

[2025-05-20 04:57:03] INFO:     127.0.0.1:37716 - "GET /get_model_info HTTP/1.1" 200 OK


## Get Server Info
Gets the server information including CLI arguments, token limits, and memory pool sizes.
- Note: `get_server_info` merges the following deprecated endpoints:
  - `get_server_args`
  - `get_memory_pool_size` 
  - `get_max_total_num_tokens`

In [4]:
# get_server_info

url = f"http://localhost:{port}/get_server_info"

response = requests.get(url)
print_highlight(response.text)

[2025-05-20 04:57:03] INFO:     127.0.0.1:37728 - "GET /get_server_info HTTP/1.1" 200 OK


## Health Check
- `/health`: Check the health of the server.
- `/health_generate`: Check the health of the server by generating one token.

In [5]:
url = f"http://localhost:{port}/health_generate"

response = requests.get(url)
print_highlight(response.text)

[2025-05-20 04:57:03] Prefill batch. #new-seq: 1, #new-token: 1, #cached-token: 0, token usage: 0.00, #running-req: 0, #queue-req: 0


[2025-05-20 04:57:04] INFO:     127.0.0.1:37742 - "GET /health_generate HTTP/1.1" 200 OK


In [6]:
url = f"http://localhost:{port}/health"

response = requests.get(url)
print_highlight(response.text)

[2025-05-20 04:57:04] INFO:     127.0.0.1:37754 - "GET /health HTTP/1.1" 200 OK


## Flush Cache

Flush the radix cache. It will be automatically triggered when the model weights are updated by the `/update_weights` API.

In [7]:
# flush cache

url = f"http://localhost:{port}/flush_cache"

response = requests.post(url)
print_highlight(response.text)

[2025-05-20 04:57:04] Cache flushed successfully!
[2025-05-20 04:57:04] INFO:     127.0.0.1:37764 - "POST /flush_cache HTTP/1.1" 200 OK


## Update Weights From Disk

Update model weights from disk without restarting the server. Only applicable for models with the same architecture and parameter size.

SGLang support `update_weights_from_disk` API for continuous evaluation during training (save checkpoint to disk and update weights from disk).


In [8]:
# successful update with same architecture and size

url = f"http://localhost:{port}/update_weights_from_disk"
data = {"model_path": "qwen/qwen2.5-0.5b-instruct"}

response = requests.post(url, json=data)
print_highlight(response.text)
assert response.json()["success"] is True
assert response.json()["message"] == "Succeeded to update model weights."

[2025-05-20 04:57:04] Start update_weights. Load format=auto
[2025-05-20 04:57:04] Update engine weights online from disk begin. avail mem=72.83 GB
[2025-05-20 04:57:04] Using model weights format ['*.safetensors']


[2025-05-20 04:57:05] No model.safetensors.index.json found in remote.
Loading safetensors checkpoint shards:   0% Completed | 0/1 [00:00<?, ?it/s]


Loading safetensors checkpoint shards: 100% Completed | 1/1 [00:00<00:00,  2.13it/s]
Loading safetensors checkpoint shards: 100% Completed | 1/1 [00:00<00:00,  2.13it/s]

[2025-05-20 04:57:05] Update weights end.
[2025-05-20 04:57:05] Cache flushed successfully!
[2025-05-20 04:57:05] INFO:     127.0.0.1:37772 - "POST /update_weights_from_disk HTTP/1.1" 200 OK


In [9]:
# failed update with different parameter size or wrong name

url = f"http://localhost:{port}/update_weights_from_disk"
data = {"model_path": "qwen/qwen2.5-0.5b-instruct-wrong"}

response = requests.post(url, json=data)
response_json = response.json()
print_highlight(response_json)
assert response_json["success"] is False
assert response_json["message"] == (
    "Failed to get weights iterator: "
    "qwen/qwen2.5-0.5b-instruct-wrong"
    " (repository not found)."
)

[2025-05-20 04:57:05] Start update_weights. Load format=auto
[2025-05-20 04:57:05] Update engine weights online from disk begin. avail mem=72.83 GB


[2025-05-20 04:57:06] Failed to get weights iterator: qwen/qwen2.5-0.5b-instruct-wrong (repository not found).
[2025-05-20 04:57:06] INFO:     127.0.0.1:37778 - "POST /update_weights_from_disk HTTP/1.1" 400 Bad Request


In [10]:
terminate_process(server_process)

## Encode (embedding model)

Encode text into embeddings. Note that this API is only available for [embedding models](openai_api_embeddings.html#openai-apis-embedding) and will raise an error for generation models.
Therefore, we launch a new server to server an embedding model.

In [11]:
embedding_process, port = launch_server_cmd(
    """
python3 -m sglang.launch_server --model-path Alibaba-NLP/gte-Qwen2-1.5B-instruct \
    --host 0.0.0.0 --is-embedding
"""
)

wait_for_server(f"http://localhost:{port}")

[2025-05-20 04:57:12] server_args=ServerArgs(model_path='Alibaba-NLP/gte-Qwen2-1.5B-instruct', tokenizer_path='Alibaba-NLP/gte-Qwen2-1.5B-instruct', tokenizer_mode='auto', skip_tokenizer_init=False, load_format='auto', trust_remote_code=False, dtype='auto', kv_cache_dtype='auto', quantization=None, quantization_param_path=None, context_length=None, device='cuda', served_model_name='Alibaba-NLP/gte-Qwen2-1.5B-instruct', chat_template=None, completion_template=None, is_embedding=True, enable_multimodal=None, revision=None, host='0.0.0.0', port=39317, mem_fraction_static=0.88, max_running_requests=200, max_total_tokens=20480, chunked_prefill_size=8192, max_prefill_tokens=16384, schedule_policy='fcfs', schedule_conservativeness=1.0, cpu_offload_gb=0, page_size=1, tp_size=1, pp_size=1, max_micro_batch_size=None, stream_interval=1, stream_output=False, random_seed=293737904, constrained_json_whitespace_pattern=None, watchdog_timeout=300, dist_timeout=None, download_dir=None, base_gpu_id=0, g

[2025-05-20 04:57:14] Downcasting torch.float32 to torch.float16.


[2025-05-20 04:57:19] Downcasting torch.float32 to torch.float16.


[2025-05-20 04:57:20] Overlap scheduler is disabled for embedding models.
[2025-05-20 04:57:20] Downcasting torch.float32 to torch.float16.


[2025-05-20 04:57:20] Attention backend not set. Use fa3 backend by default.
[2025-05-20 04:57:20] Init torch distributed begin.


...

[2025-05-20 04:57:28] max_total_num_tokens=20480, chunked_prefill_size=8192, max_prefill_tokens=16384, max_running_requests=200, context_len=131072


[2025-05-20 04:57:28] INFO:     Started server process [400813]
[2025-05-20 04:57:28] INFO:     Waiting for application startup.
[2025-05-20 04:57:28] INFO:     Application startup complete.
[2025-05-20 04:57:28] INFO:     Uvicorn running on http://0.0.0.0:39317 (Press CTRL+C to quit)


[2025-05-20 04:57:29] INFO:     127.0.0.1:43760 - "GET /v1/models HTTP/1.1" 200 OK


[2025-05-20 04:57:29] INFO:     127.0.0.1:43776 - "GET /get_model_info HTTP/1.1" 200 OK
[2025-05-20 04:57:29] Prefill batch. #new-seq: 1, #new-token: 6, #cached-token: 0, token usage: 0.00, #running-req: 0, #queue-req: 0


[2025-05-20 04:57:30] INFO:     127.0.0.1:43792 - "POST /encode HTTP/1.1" 200 OK
[2025-05-20 04:57:30] The server is fired up and ready to roll!


In [12]:
# successful encode for embedding model

url = f"http://localhost:{port}/encode"
data = {"model": "Alibaba-NLP/gte-Qwen2-1.5B-instruct", "text": "Once upon a time"}

response = requests.post(url, json=data)
response_json = response.json()
print_highlight(f"Text embedding (first 10): {response_json['embedding'][:10]}")

[2025-05-20 04:57:34] Prefill batch. #new-seq: 1, #new-token: 4, #cached-token: 0, token usage: 0.00, #running-req: 0, #queue-req: 0
[2025-05-20 04:57:34] INFO:     127.0.0.1:34324 - "POST /encode HTTP/1.1" 200 OK


In [13]:
terminate_process(embedding_process)

## Classify (reward model)

SGLang Runtime also supports reward models. Here we use a reward model to classify the quality of pairwise generations.

In [14]:
terminate_process(embedding_process)

# Note that SGLang now treats embedding models and reward models as the same type of models.
# This will be updated in the future.

reward_process, port = launch_server_cmd(
    """
python3 -m sglang.launch_server --model-path Skywork/Skywork-Reward-Llama-3.1-8B-v0.2 --host 0.0.0.0 --is-embedding
"""
)

wait_for_server(f"http://localhost:{port}")

[2025-05-20 04:57:41] server_args=ServerArgs(model_path='Skywork/Skywork-Reward-Llama-3.1-8B-v0.2', tokenizer_path='Skywork/Skywork-Reward-Llama-3.1-8B-v0.2', tokenizer_mode='auto', skip_tokenizer_init=False, load_format='auto', trust_remote_code=False, dtype='auto', kv_cache_dtype='auto', quantization=None, quantization_param_path=None, context_length=None, device='cuda', served_model_name='Skywork/Skywork-Reward-Llama-3.1-8B-v0.2', chat_template=None, completion_template=None, is_embedding=True, enable_multimodal=None, revision=None, host='0.0.0.0', port=37781, mem_fraction_static=0.88, max_running_requests=200, max_total_tokens=20480, chunked_prefill_size=8192, max_prefill_tokens=16384, schedule_policy='fcfs', schedule_conservativeness=1.0, cpu_offload_gb=0, page_size=1, tp_size=1, pp_size=1, max_micro_batch_size=None, stream_interval=1, stream_output=False, random_seed=990680944, constrained_json_whitespace_pattern=None, watchdog_timeout=300, dist_timeout=None, download_dir=None, b

[2025-05-20 04:57:49] Overlap scheduler is disabled for embedding models.


[2025-05-20 04:57:52] Attention backend not set. Use flashinfer backend by default.
[2025-05-20 04:57:52] Init torch distributed begin.


[2025-05-20 04:57:53] Init torch distributed ends. mem usage=0.00 GB
[2025-05-20 04:57:53] init_expert_location from trivial


[2025-05-20 04:57:54] Load weight begin. avail mem=62.21 GB


...

2025-05-20 04:58:52,344 - INFO - flashinfer.jit: Prebuilt kernels not found, using JIT backend


[2025-05-20 04:58:53] max_total_num_tokens=20480, chunked_prefill_size=8192, max_prefill_tokens=16384, max_running_requests=200, context_len=131072


[2025-05-20 04:58:53] INFO:     Started server process [403052]
[2025-05-20 04:58:53] INFO:     Waiting for application startup.
[2025-05-20 04:58:53] INFO:     Application startup complete.
[2025-05-20 04:58:53] INFO:     Uvicorn running on http://0.0.0.0:37781 (Press CTRL+C to quit)


[2025-05-20 04:58:54] INFO:     127.0.0.1:38236 - "GET /v1/models HTTP/1.1" 200 OK
[2025-05-20 04:58:54] INFO:     127.0.0.1:38250 - "GET /get_model_info HTTP/1.1" 200 OK
[2025-05-20 04:58:54] Prefill batch. #new-seq: 1, #new-token: 7, #cached-token: 0, token usage: 0.00, #running-req: 0, #queue-req: 0


2025-05-20 04:58:55,112 - INFO - flashinfer.jit: Loading JIT ops: batch_prefill_with_kv_cache_dtype_q_bf16_dtype_kv_bf16_dtype_o_bf16_dtype_idx_i32_head_dim_qk_128_head_dim_vo_128_posenc_0_use_swa_False_use_logits_cap_False_f16qk_False_sm90


In [15]:
from transformers import AutoTokenizer

PROMPT = (
    "What is the range of the numeric output of a sigmoid node in a neural network?"
)

RESPONSE1 = "The output of a sigmoid node is bounded between -1 and 1."
RESPONSE2 = "The output of a sigmoid node is bounded between 0 and 1."

CONVS = [
    [{"role": "user", "content": PROMPT}, {"role": "assistant", "content": RESPONSE1}],
    [{"role": "user", "content": PROMPT}, {"role": "assistant", "content": RESPONSE2}],
]

tokenizer = AutoTokenizer.from_pretrained("Skywork/Skywork-Reward-Llama-3.1-8B-v0.2")
prompts = tokenizer.apply_chat_template(CONVS, tokenize=False)

url = f"http://localhost:{port}/classify"
data = {"model": "Skywork/Skywork-Reward-Llama-3.1-8B-v0.2", "text": prompts}

responses = requests.post(url, json=data).json()
for response in responses:
    print_highlight(f"reward: {response['embedding'][0]}")

2025-05-20 04:59:45,172 - INFO - flashinfer.jit: Finished loading JIT ops: batch_prefill_with_kv_cache_dtype_q_bf16_dtype_kv_bf16_dtype_o_bf16_dtype_idx_i32_head_dim_qk_128_head_dim_vo_128_posenc_0_use_swa_False_use_logits_cap_False_f16qk_False_sm90
2025-05-20 04:59:45,179 - INFO - flashinfer.jit: Loading JIT ops: batch_prefill_with_kv_cache_dtype_q_bf16_dtype_kv_bf16_dtype_o_bf16_dtype_idx_i32_head_dim_qk_128_head_dim_vo_128_posenc_0_use_swa_False_use_logits_cap_False_f16qk_False
2025-05-20 04:59:45,198 - INFO - flashinfer.jit: Finished loading JIT ops: batch_prefill_with_kv_cache_dtype_q_bf16_dtype_kv_bf16_dtype_o_bf16_dtype_idx_i32_head_dim_qk_128_head_dim_vo_128_posenc_0_use_swa_False_use_logits_cap_False_f16qk_False


[2025-05-20 04:59:45] Prefill batch. #new-seq: 2, #new-token: 136, #cached-token: 2, token usage: 0.00, #running-req: 0, #queue-req: 0
[2025-05-20 04:59:45] INFO:     127.0.0.1:38262 - "POST /encode HTTP/1.1" 200 OK
[2025-05-20 04:59:45] The server is fired up and ready to roll!
2025-05-20 04:59:45,404 - INFO - flashinfer.jit: Loading JIT ops: cascade
2025-05-20 04:59:45,423 - INFO - flashinfer.jit: Finished loading JIT ops: cascade
[2025-05-20 04:59:45] INFO:     127.0.0.1:38264 - "POST /classify HTTP/1.1" 200 OK


In [16]:
terminate_process(reward_process)

## Capture expert selection distribution in MoE models

SGLang Runtime supports recording the number of times an expert is selected in a MoE model run for each expert in the model. This is useful when analyzing the throughput of the model and plan for optimization.

*Note: We only print out the first 10 lines of the csv below for better readability. Please adjust accordingly if you want to analyze the results more deeply.*

In [17]:
expert_record_server_process, port = launch_server_cmd(
    "python3 -m sglang.launch_server --model-path Qwen/Qwen1.5-MoE-A2.7B --host 0.0.0.0 --expert-distribution-recorder-mode stat"
)

wait_for_server(f"http://localhost:{port}")

[2025-05-20 04:59:52] server_args=ServerArgs(model_path='Qwen/Qwen1.5-MoE-A2.7B', tokenizer_path='Qwen/Qwen1.5-MoE-A2.7B', tokenizer_mode='auto', skip_tokenizer_init=False, load_format='auto', trust_remote_code=False, dtype='auto', kv_cache_dtype='auto', quantization=None, quantization_param_path=None, context_length=None, device='cuda', served_model_name='Qwen/Qwen1.5-MoE-A2.7B', chat_template=None, completion_template=None, is_embedding=False, enable_multimodal=None, revision=None, host='0.0.0.0', port=35346, mem_fraction_static=0.88, max_running_requests=200, max_total_tokens=20480, chunked_prefill_size=8192, max_prefill_tokens=16384, schedule_policy='fcfs', schedule_conservativeness=1.0, cpu_offload_gb=0, page_size=1, tp_size=1, pp_size=1, max_micro_batch_size=None, stream_interval=1, stream_output=False, random_seed=91543216, constrained_json_whitespace_pattern=None, watchdog_timeout=300, dist_timeout=None, download_dir=None, base_gpu_id=0, gpu_id_step=1, log_level='info', log_lev

[2025-05-20 05:00:00] Attention backend not set. Use flashinfer backend by default.
[2025-05-20 05:00:00] Init torch distributed begin.


[2025-05-20 05:00:01] Init torch distributed ends. mem usage=0.00 GB
[2025-05-20 05:00:01] init_expert_location from trivial


[2025-05-20 05:00:02] Load weight begin. avail mem=61.89 GB


[2025-05-20 05:00:03] Using model weights format ['*.safetensors']


...

[2025-05-20 05:01:07] INFO:     Started server process [408634]
[2025-05-20 05:01:07] INFO:     Waiting for application startup.
[2025-05-20 05:01:07] INFO:     Application startup complete.
[2025-05-20 05:01:07] INFO:     Uvicorn running on http://0.0.0.0:35346 (Press CTRL+C to quit)


[2025-05-20 05:01:08] INFO:     127.0.0.1:54044 - "GET /v1/models HTTP/1.1" 200 OK
[2025-05-20 05:01:08] INFO:     127.0.0.1:54056 - "GET /get_model_info HTTP/1.1" 200 OK
[2025-05-20 05:01:08] Prefill batch. #new-seq: 1, #new-token: 6, #cached-token: 0, token usage: 0.00, #running-req: 0, #queue-req: 0


2025-05-20 05:01:09,807 - INFO - flashinfer.jit: Loading JIT ops: batch_prefill_with_kv_cache_dtype_q_bf16_dtype_kv_bf16_dtype_o_bf16_dtype_idx_i32_head_dim_qk_128_head_dim_vo_128_posenc_0_use_swa_False_use_logits_cap_False_f16qk_False_sm90
2025-05-20 05:01:09,830 - INFO - flashinfer.jit: Finished loading JIT ops: batch_prefill_with_kv_cache_dtype_q_bf16_dtype_kv_bf16_dtype_o_bf16_dtype_idx_i32_head_dim_qk_128_head_dim_vo_128_posenc_0_use_swa_False_use_logits_cap_False_f16qk_False_sm90
2025-05-20 05:01:09,837 - INFO - flashinfer.jit: Loading JIT ops: batch_prefill_with_kv_cache_dtype_q_bf16_dtype_kv_bf16_dtype_o_bf16_dtype_idx_i32_head_dim_qk_128_head_dim_vo_128_posenc_0_use_swa_False_use_logits_cap_False_f16qk_False
2025-05-20 05:01:09,857 - INFO - flashinfer.jit: Finished loading JIT ops: batch_prefill_with_kv_cache_dtype_q_bf16_dtype_kv_bf16_dtype_o_bf16_dtype_idx_i32_head_dim_qk_128_head_dim_vo_128_posenc_0_use_swa_False_use_logits_cap_False_f16qk_False


[2025-05-20 05:01:10] Using default MoE kernel config. Performance might be sub-optimal! Config file not found at /public_sglang_ci/runner-kc-gpu-1/_work/sglang/sglang/python/sglang/srt/layers/moe/fused_moe_triton/configs/E=60,N=1408,device_name=NVIDIA_H100_80GB_HBM3.json, you can create them with https://github.com/sgl-project/sglang/tree/main/benchmark/kernels/fused_moe_triton


2025-05-20 05:01:10,303 - INFO - flashinfer.jit: Loading JIT ops: batch_decode_with_kv_cache_dtype_q_bf16_dtype_kv_bf16_dtype_o_bf16_dtype_idx_i32_head_dim_qk_128_head_dim_vo_128_posenc_0_use_swa_False_use_logits_cap_False


In [18]:
response = requests.post(f"http://localhost:{port}/start_expert_distribution_record")
print_highlight(response)

url = f"http://localhost:{port}/generate"
data = {"text": "What is the capital of France?"}

response = requests.post(url, json=data)
print_highlight(response.json())

response = requests.post(f"http://localhost:{port}/stop_expert_distribution_record")
print_highlight(response)

response = requests.post(f"http://localhost:{port}/dump_expert_distribution_record")
print_highlight(response)

2025-05-20 05:01:25,623 - INFO - flashinfer.jit: Finished loading JIT ops: batch_decode_with_kv_cache_dtype_q_bf16_dtype_kv_bf16_dtype_o_bf16_dtype_idx_i32_head_dim_qk_128_head_dim_vo_128_posenc_0_use_swa_False_use_logits_cap_False
[2025-05-20 05:01:25] Resetting ExpertDistributionRecorder...
[2025-05-20 05:01:25] INFO:     127.0.0.1:48080 - "POST /start_expert_distribution_record HTTP/1.1" 200 OK


[2025-05-20 05:01:25] Prefill batch. #new-seq: 1, #new-token: 7, #cached-token: 0, token usage: 0.00, #running-req: 1, #queue-req: 0


[2025-05-20 05:01:25] INFO:     127.0.0.1:40640 - "POST /generate HTTP/1.1" 200 OK


[2025-05-20 05:01:25] INFO:     127.0.0.1:40650 - "POST /stop_expert_distribution_record HTTP/1.1" 200 OK


[2025-05-20 05:01:26] Write expert distribution to /tmp/expert_distribution_recorder_1747717286.5672863.pt
[2025-05-20 05:01:26] Resetting ExpertDistributionRecorder...
[2025-05-20 05:01:26] INFO:     127.0.0.1:40666 - "POST /dump_expert_distribution_record HTTP/1.1" 200 OK


In [19]:
terminate_process(expert_record_server_process)